this is for blind implement RAG concept on medium blog by AJ Aey.

In [1]:
%pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.6/982.6 kB 15.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install llama-index-readers-elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install llama-index-vector-stores-opensearch

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install llama-index-embeddings-ollama

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 25.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 24.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 21.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 18.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


now preparation section has completed.

In [1]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
import nest_asyncio
from os import getenv

# Apply nest_asyncio to avoid runtime errors in Jupyter notebooks
nest_asyncio.apply()

# Check if CUDA is available for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/Volumes/SS2305_1MH01/anaconda-env/kdai-llm-final/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


now setup hybrid search pipeline on opensearch.
to run this code, ensure opensearch container is running.

In [2]:
! curl -XPUT "http://localhost:9200/_search/pipeline/hybrid-search-pipeline" -H 'Content-Type: application/json' -d' \
{ \
  "description": "Pipeline for hybrid search", \
  "phase_results_processors": [ \
    { \
      "normalization-processor": { \
        "normalization": { \
          "technique": "min_max" \
        }, \
        "combination": { \
          "technique": "harmonic_mean", \
          "parameters": { \
            "weights": [ \
              0.3, \
              0.7 \
            ] \
          } \
        } \
      } \
    } \
  ] \
}'

{"acknowledged":true}

3. Load and process PDF documents
In this step, we’ll load PDF documents and convert them into a format suitable for further processing, using Llama Index to help manage the data.

3.1 Load PDF documents:
Explanation: We use SimpleDirectoryReader to load PDF files from a specified directory. This function is part of Llama Index and is designed to handle various document formats, including PDFs. It automatically extracts text content from the PDFs, making it easier to process the information in subsequent steps.

PDF file containing the article 291 judgement will be placed in this folder.
it will not be part of opensearch container. it will also not sync to github.
this data source will be compressed and backup on onedrive instead of github.

In [3]:
# Specify the path to the folder on the external disk
external_disk_path = "/Volumes/SS2305_1MH01/data-src/kdai-llm-final-20241007/pdf_corpus"  # macOS/Linux
# external_disk_path = "D:/pdf_corpus"  # Windows

In [4]:
from llama_index.core import SimpleDirectoryReader
#reader = SimpleDirectoryReader(input_dir="pdf_corpus",recursive=True)
reader = SimpleDirectoryReader(input_dir=external_disk_path,recursive=True)
documents = reader.load_data()

3.2 Split documents into chunks:

In [5]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separator=" ",
)
token_nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes: 100%|██████████| 88/88 [00:00<00:00, 310.89it/s]


4. Generate embeddings using the BAAI/bge-m3 model
In this step, we’ll use the BAAI/bge-m3 model to generate embeddings for our text chunks. These embeddings are crucial for semantic search and understanding the content of our documents.

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = 'BAAI/bge-m3'
embedding_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=512, device=device)

In [7]:
embeddings = embedding_model.get_text_embedding("box")
dim = len(embeddings)
print("embedding dimension of example text ===>",dim)

embedding dimension of example text ===> 1024


5. Setup and Initialize OpenSearch Vector Client
In this step, we’ll set up and initialize the OpenSearch Vector Client, preparing our system to use OpenSearch as a vector database for efficient similarity searches in the future.

5.1 Set up OpenSearch connection details

In [8]:
from os import getenv
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "http://localhost:9200")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "test_pdf_index")

5.2 Configure OpenSearchVectorClient

In [9]:
# OpensearchVectorClient stores text in this field by default
text_field = "content_text"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled with hybrid search pipeline
client = OpensearchVectorClient(
    endpoint=endpoint,
    index=idx,
    dim=dim,
    embedding_field=embedding_field,
    text_field=text_field,
    search_pipeline="hybrid-search-pipeline",
)

5.3 Initialize OpensearchVectorStore
We create an instance of OpensearchVectorStore using the configured client. This provides an interface for managing the vector store within the Llama Index framework, facilitating easy integration with other Llama Index functionalities.

In [10]:
# initialize vector store
vector_store = OpensearchVectorStore(client)

6. Create VectorStoreIndex and Store Embeddings
In this step, we’ll create a VectorStoreIndex using Llama Index and explicitly store our document embeddings in OpenSearch. This process enables efficient semantic search and retrieval of information from our processed documents.

6.1 Create a StorageContext
Here, we create a StorageContext using the vector_store (OpenSearch) we initialized in the previous step. This StorageContext provides a standardized interface for managing storage in Llama Index and connects our index to the OpenSearch vector store.

In [11]:
from llama_index.core import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

6.2 Create VectorStoreIndex, Generate and Store Embeddings
In this crucial step, we create the VectorStoreIndex, generate embeddings for our documents, and store them in OpenSearch:

In [12]:
index = VectorStoreIndex(
    token_nodes, storage_context=storage_context, embed_model=embedding_model
)

7. Use the index to answer questions about the PDF content
In this section, we’ll use our created index to answer questions about the content of our PDF documents.

7.1 Set up the retriever
We create a retriever from our index with the following parameters:

similarity_top_k=3: This specifies that we want to retrieve the top 3 most similar results.

vector_store_query_mode=VectorStoreQueryMode.HYBRID: This enables hybrid search, combining both keyword and semantic search for better results.

In [13]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)

7.2 Define the query
This is the question we want to ask about our PDF content.

this part we should use our own question. it is the question regarding legal principle from article 291 criminal law.

question is deliberately not mention to article 291 in criminal law. we wan to see whether the model can understand this and retrive the correct document.

In [38]:
#question = "นายดำเข้าแย่งมีดจากนายแดง และนายดำถูกแทงเสียชีวิต หากข้อเท็จจริงฟังได้ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย นายแดงจะอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้หรือไม่ ศาลจะวินิจฉัยอย่างไร"
question = "นายดำเข้าแย่งมีดจากนายแดง และนายดำถูกแทงเสียชีวิต หากข้อเท็จจริงฟังได้ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย นายแดงจะอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้หรือไม่ เพราะอะไร"

In [14]:
question2 = "นายดำเข้าแย่งมีดจากนายแดง และนายดำถูกแทงเสียชีวิต หากข้อเท็จจริงฟังได้ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย นายแดงจะอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้หรือไม่ เพราะอะไร ให้ยกตัวอย่างคำพิพากษาศาลฎีกาหรือหมายเลขคดีแดงเรื่องนี้ด้วย"

7.3 Retrieve relevant information
We use the retriever to find the most relevant information from our indexed documents based on the query.

In [39]:
prompt = retriever.retrieve(question)

In [15]:
prompt2 = retriever.retrieve(question2)

7.4 Display the results
We iterate through the retrieved results, printing both the metadata and the content of each retrieved chunk.

This step demonstrates how we can use our indexed documents to retrieve relevant information based on a natural language query. The hybrid search mode allows us to leverage both keyword matching and semantic similarity, potentially providing more accurate and contextually relevant results.

By using this approach, we can efficiently answer questions about the content of our PDF documents, even if the exact wording doesn’t match what’s in the documents. This is particularly useful for creating intelligent document querying systems or chatbots that can understand and respond to questions about specific document contents.

In [16]:
for r in prompt:
    print(r.metadata)
    print(r)

NameError: name 'prompt' is not defined

In [17]:
for r in prompt2:
    print(r.metadata)
    print(r)

{'page_label': '1', 'file_name': '1597_2562.pdf', 'file_path': '/Volumes/SS2305_1MH01/data-src/kdai-llm-final-20241007/pdf_corpus/1597_2562.pdf', 'file_type': 'application/pdf', 'file_size': 124005, 'creation_date': '2024-10-08', 'last_modified_date': '2024-10-03'}
Node ID: b4d6695e-9edd-4456-96a5-f0423e75d7fe
Text:  กา ด นเ ยว บ ญหา อกฎหมาย   พากษาศาล กา 
1597/2562พ กงาน ยการ งห ด มพร โจท  นาย ส .  เลย ป.อ. มาตรา  68, 288,
291 การกระ  งจะเ นการ อง นโดยชอบ วยกฎหมายตาม  ป . อ . มาตรา  68  อง
เ นการกระ โดยเจตนา   เลยเอาอา ธ นออกมา  ตายและ  น นโดยประมาท
 ก ตาย งแ ความตายไ ใ การกระ โดยเจตนา  การกระ ของ เลย งไ ใ การ  อง น
___________________________ โจท  องขอใ ...
Score:  1.000

{'page_label': '5', 'file_name': '1597_2562.pdf', 'file_path': '/Volumes/SS2305_1MH01/data-src/kdai-llm-final-20241007/pdf_corpus/1597_2562.pdf', 'file_type': 'application/pdf', 'file_size': 124005, 'creation_date': '2024-10-08', 'last_modified_date': '2024-10-03'}
Node ID: b1a95e97-af8f-4d57-a2c2-891640e53402
Text:

8. Implement Few-Shot Learning with OpenThaiGPT on Ollama

8.1 Set up Ollama with OpenThaiGPT

In [32]:
!ollama search openthaigpt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error: unknown command "search" for "ollama"


In [31]:
!ollama pull openthaigpt:latest


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
Error: pull model manifest: file does not exist


In [33]:
! ollama list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NAME               ID              SIZE      MODIFIED      
llama3.2:latest    a80c4f17acd5    2.0 GB    7 minutes ago    


8.2 Create a function to query OpenThaiGPT
This function takes the user’s question and the retrieved context, formats them into a prompt, and sends it to the OpenThaiGPT model running on Ollama.

In [51]:
"""
#this is example from Aj. we skip this part and use our own prompt regarding article 291 in the next cell.
import ollama

def query_openthaigpt(question, context):
    formatted_prompt = f'''# Few-shot examples
Example 1:
Context: An ONT (Optical Network Terminal) device is unable to connect to the network.
Question: What are the steps to troubleshoot an ONT that cannot connect?
Answer: 1. Check the fiber optic cable connection.
2. Restart the ONT device.
3. Verify the status lights on the device.
4. Test the connection with a spare ONT.
5. Contact technical support if the issue persists.

Example 2:
Context: A customer complains about slow internet speed in their FTTX system.
Question: What is the procedure for diagnosing and fixing slow internet speed in an FTTX system?
Answer: 1. Run an internet speed test using a speed testing tool.
2. Check the Wi-Fi router settings.
3. Test the connection directly to the ONT using an Ethernet cable.
4. Verify the optical signal strength at the ONT.
5. Check the customer's bandwidth usage.
6. Adjust device settings or replace faulty equipment if necessary.

# RAG component
Retrieved information:
{context}

# Actual question
Question: {question}

Please answer the question with Thai language using the information from the examples and the retrieved information above. Focus on troubleshooting end-user devices in FTTX projects. Provide a clear, step-by-step answer, prioritized in order of importance. Include a brief explanation for each step. If the provided information is insufficient to answer the question completely, state "The available information is not sufficient to fully answer this question" and suggest general troubleshooting steps.

Answer:'''

    #print(formatted_prompt)
    response = ollama.generate(model='openthaigpt:latest', prompt=formatted_prompt)
    return response['response']

"""

'\n#this is example from Aj. we skip this part and use our own prompt regarding article 291 in the next cell.\nimport ollama\n\ndef query_openthaigpt(question, context):\n    formatted_prompt = f\'\'\'# Few-shot examples\nExample 1:\nContext: An ONT (Optical Network Terminal) device is unable to connect to the network.\nQuestion: What are the steps to troubleshoot an ONT that cannot connect?\nAnswer: 1. Check the fiber optic cable connection.\n2. Restart the ONT device.\n3. Verify the status lights on the device.\n4. Test the connection with a spare ONT.\n5. Contact technical support if the issue persists.\n\nExample 2:\nContext: A customer complains about slow internet speed in their FTTX system.\nQuestion: What is the procedure for diagnosing and fixing slow internet speed in an FTTX system?\nAnswer: 1. Run an internet speed test using a speed testing tool.\n2. Check the Wi-Fi router settings.\n3. Test the connection directly to the ONT using an Ethernet cable.\n4. Verify the optical

ถาม: นาย A ฆ่าคนตายโดยอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม ป.อาญา ม.68 หากข้อเท็จจริงฟังได้ว่ามีการยื้อแย่งอาวุธปืนและอาวุธปืนลั่น ศาลจะวินิจฉัยอย่างไร

ตอบ: ศาลจะตดสินว่าคำให้การของจำเลยฟังไม่ขึ้น ด้วยว่าการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม ม.68 จะต้องเป็นการกระทำโดยเจตนา เมื่อข้อเท็จจริงฟังได้ว่ามีการยื้อแย่งปืน และปืนลั่น จึงเป็นการกระทำโดยประมาท เป็นเหตุให้ถึงแก่ความตาย ไม่ใช่การกระทำโดยเจตนา ไม่เป็นการป้องกันตัวโดยชอบด้วยกฎหมาย

#english text translated by Claude AI 3.5 sonnet.
Question: Mr. A killed a person and claimed it was justifiable self-defense under Section 68 of the Criminal Code. If the facts indicate that there was a struggle for a firearm and the gun accidentally discharged, how would the court rule?
Answer: The court would likely determine that the defendant's claim of self-defense is not tenable. Justifiable self-defense under Section 68 requires intentional action. Given that the facts indicate a struggle for the firearm and an accidental discharge, this constitutes a negligent act resulting in death, not an intentional act. Therefore, it does not meet the criteria for justifiable self-defense under the law.
The court would likely rule that this case falls under negligent homicide rather than justifiable self-defense, as the element of intent, which is crucial for the self-defense claim, is absent in the scenario where the firearm discharged accidentally during a struggle.

In [18]:
import ollama

def query_openthaigpt(question, context):
    formatted_prompt = f'''# Few-shot examples
Example 1:
Context: Mr. A killed a person and claimed it was justifiable self-defense under Section 68 of the Criminal Code.
If the facts indicate that there was a struggle for a firearm and the gun accidentally discharged.
Question: How would the court rule? Please provide an example Supreme Court decision case number or the Red case number (หมายเลขคดีแดง)
Answer: The court would likely determine that the defendant's claim of self-defense is not tenable. Justifiable self-defense under Section 68 requires intentional action. Given that the facts indicate a struggle for the firearm and an accidental discharge, this constitutes a negligent act resulting in death, not an intentional act. Therefore, it does not meet the criteria for justifiable self-defense under the law.
The court would likely rule that this case falls under negligent homicide rather than justifiable self-defense,
as the element of intent, which is crucial for the self-defense claim,
is absent in the scenario where the firearm discharged accidentally during a struggle.
The example Supreme Court decision is Case No. 1597/2562, Red Case No. อ632/2560.

# RAG component
Retrieved information:
{context}

# Actual question
Question: {question}

Please answer the question with Thai language using the information from the examples and the retrieved information above. Focus on troubleshooting end-user devices in FTTX projects. Provide a clear, step-by-step answer, prioritized in order of importance. Include a brief explanation for each step. If the provided information is insufficient to answer the question completely, state "The available information is not sufficient to fully answer this question" and suggest general troubleshooting steps.

Answer:'''

    #print(formatted_prompt)
    #response = ollama.generate(model='openthaigpt:latest', prompt=formatted_prompt)
    response = ollama.generate(model='llama3.2:latest', prompt=formatted_prompt)
    return response['response']

8.3 Integrate OpenThaiGPT with our retriever
Now, let’s modify our question-answering process to use OpenThaiGPT:

This function retrieves relevant information using our previously set up retriever, combines the retrieved text into a context, and then uses OpenThaiGPT to generate an answer based on this context.

In [42]:
def answer_question(question):   
    # Query OpenThaiGPT
    answer = query_openthaigpt(question, prompt[0])
    return answer

# Example usage
answer = answer_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: นายดำเข้าแย่งมีดจากนายแดง และนายดำถูกแทงเสียชีวิต หากข้อเท็จจริงฟังได้ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย นายแดงจะอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้หรือไม่ เพราะอะไร
Answer: 1. นายแดงจะไม่สามารถอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้ เนื่องจากการกระทำของนายแดงเป็นการตอบสนองต่อการถูกโจมตี ซึ่งไม่ค่อยได้รับการยอมรับในข้อเท็จจริงฟังได้ว่านายแดงสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญ
2. การใช้มาตรา 68 ต้องอาศัยการกระทำที่เกิดขึ้นโดยเจตนา ซึ่งนอกจากการป้องกันตัวในกรณีนี้ ยังมีการโจมตีของนายแดงด้วยมีด 
3. ในการขยายความตาม มาตรา 288 และ 291 โดยอาจพิจารณาคดีเป็นการเสียชีวิตอย่างไม่สมเหตุสมผลหรือโดยประมาท เนื่องจากสามารถสันนิษฐานได้ว่ามีนัยยะของการโจมตีที่รุนแรงมากขึ้นซึ่งล้ำเกณฑ์ความจำเป็นในการป้องกันตัว
4. ดังนั้น การอ้างว่าเป็นการป้องกันตัวตาม มาตรา 68 จะต้องมีการพิสูจน์ที่แน่นอนและสมเหตุสมผลมากกว่านี้


In [19]:
def answer_question(question2):   
    # Query OpenThaiGPT
    answer = query_openthaigpt(question2, prompt2[0])
    return answer

# Example usage
answer = answer_question(question2)
print(f"Question: {question2}")
print(f"Answer: {answer}")

Question: นายดำเข้าแย่งมีดจากนายแดง และนายดำถูกแทงเสียชีวิต หากข้อเท็จจริงฟังได้ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย นายแดงจะอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้หรือไม่ เพราะอะไร ให้ยกตัวอย่างคำพิพากษาศาลฎีกาหรือหมายเลขคดีแดงเรื่องนี้ด้วย
Answer: การอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 กฎหมายแพทย์ไม่สามารถใช้ได้ในกรณีนี้ เนื่องจากข้อเท็จจริงที่ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย ซึ่งไม่ใช่การกระทำโดยเจตนา แต่เป็นการกระทำโดยประมาท

การป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ต้องมีองค์ประกอบของการกระทำโดยเจตนาของผู้กระทำ และไม่มีการกระทำโดยเจตนาของตนเองในการป้องกันตัวในกรณีนี้ ไม่มีการกระทำที่เป็นการป้องกันตัวอย่างจริงจัง

ดังนั้น นายแดงไม่สามารถอ้างว่าเป็นการป้องกันตัวโดยชอบด้วยกฎหมาย ตาม มาตรา 68 ได้ หากข้อเท็จจริงฟังได้ว่านายดำสดุดเท้าตัวเองแล้วล้มมาโดนมีดแทงจุดสำคัญเป็นเหตุให้ถึงแก่ความตาย

หมายเลขคดีแดงเรื่องนี้ไม่ได้กล่าวไว้ในข้อมูลที่ให้มา
